### 0. Imports and requirements

* В данном соревновании мы имеем дело с последовательностями, один из интуитивных способов работы с ними - использование рекуррентных сетей. Данный бейзлайн посвящен тому, чтобы показать, как можно строить хорошие решения без использования сложного и трудоемкого feature engineering-а (чтобы эффективно решать ту же задачу с высоким качеством с помощью бустингов нужно несколько тысяч признаков), благодаря рекуррентным сетям. В этом ноутбуке мы построим решение с использованием фреймфорка `tensorflow`. Для комфортной работы Вам понадобится машина с `GPU` (хватит ресурсов `google colab` или `kaggle`).

In [1]:
%load_ext autoreload
%autoreload 2

import os
import pandas as pd
import sys
import pickle
import numpy as np

from tensorflow import keras
from tensorflow.keras import layers as L
from tensorflow.keras import Model

from sklearn.model_selection import train_test_split
from tqdm import tqdm

os.environ["CUDA_VISIBLE_DEVICES"] = '1'
pd.set_option('display.max_columns', None)

# добавим корневую папку, в ней лежат все необходимые полезные функции для обработки данных
sys.path.append('../../')
sys.path.append('../')

### 1. Data Preprocessing

In [2]:
TRAIN_TRANSACTIONS_PATH = '../../../train_transactions_contest/'
TEST_TRANSACTIONS_PATH = '../../../test_transactions_contest/'

TRAIN_TARGET_PATH = '../../../train_target.csv'

In [3]:
target_frame = pd.read_csv(TRAIN_TARGET_PATH)
target_frame.head()

,app_id,product,flag
0,0,3,0
1,1,1,0
2,2,1,0
3,3,1,0
4,4,1,0


* Как и в случае с бустингами, мы не можем поместить всю выборку в память, в виду, например, ограниченных ресурсов. Для итеративного чтения данных нам потребуется функция `utils.read_parquet_dataset_from_local`, которая читает N частей датасета за раз в память.


* Нейронные сети требуют отделнього внимания к тому, как будут поданы и обработаны данные. Важные моменты, на которые требуется обратить внимание:
    * Использование рекуррентных сетей подразумевает работу на уровне последовательностей, где одна последовательность - все исторические транзакции клиента. Чтобы преобразовать `pd.DataFrame` с транзакциями клиентов в табличном виде к последовательностям, мы подготовили функцию `dataset_preprocessing_utils.transform_transactions_to_sequences`, она делает необходимые манипуляции и возвращает фрейм с двумя колонками: `app_id` и `sequences`. Колонка `sequence` представляет из себя массив массивов длины `len(features)`, где каждый вложенный массив - значения одного конкретного признака во всех транзакциях клиента. 
    
    * каждый клиент имеет различную по длине историю транзакций. При этом обучение сетей происходит батчами, что требует делать паддинги в последовательностях. Довольно неэффективно делать паддинг внутри батча на последовательностях случайной длины (довольно часто будем делать большой и бесполезный паддинг). Гораздо лучше использовать технику `sequence_bucketing` (о ней рассказано в образовательном ролике к данному бейзлайну). Для этого мы предоставляем функцию `dataset_preprocessing_utils.create_padded_buckets`. Один из аргументов в данную функцию - `bucket_info` - словарь, где для конкретной длины последовательности указано до какой длины нужно делать паддинг. Мы предоставялем для старта простой вид разбиения на 100 бакетов и файл где лежит отображение каждой длины в падднг (файл `buckets_info.pkl`).
    
    * Такие признаки, как [`amnt`, `days_before`, `hour_diff`] по своей природе не являются категориальными. Вы в праве самостоятельно выбирать способ работы с ними (модифицируя функции бейзлайна или адаптируя под себя). В рамках бейзлайна мы предлагаем интерпретировать каждую не категориальную фичу как категориальную. Для этого нужно подготовить bin-ы для каждой фичи. Мы предлагаем простой способ разбиения по бинам.

In [4]:
from utils import read_parquet_dataset_from_local
from dataset_preprocessing_utils import transform_transactions_to_sequences, create_padded_buckets

In [5]:
import pickle

with open('../constants/buckets_info.pkl', 'rb') as f:
    mapping_seq_len_to_padded_len = pickle.load(f)
    
with open('../constants/dense_features_buckets.pkl', 'rb') as f:
    dense_features_buckets = pickle.load(f)

* Функция `create_buckets_from_transactions` ниже реализует следующий набор действий:
    * Читает `num_parts_to_preprocess_at_once` частей датасета в память
    * Преобразует вещественные и численные признаки к категориальным (используя `np.digitize` и подготовленные бины)
    * Формирует фрейм с транзакциями в виде последовательностей с помощью `transform_transactions_to_sequences`.
    * Если указан `frame_with_ids`, то использует `app_id` из `frame_with_ids` - актуально, чтобы выделить валидационную выборку.
    * Реализует технику `sequence_bucketing` и сохраняет словарь обработанных последовательностей в `.pkl` файл

In [6]:
def create_buckets_from_transactions(path_to_dataset, save_to_path, frame_with_ids = None, 
                                     num_parts_to_preprocess_at_once: int = 1, 
                                     num_parts_total=50, has_target=False):
    block = 0
    for step in tqdm(range(0, num_parts_total, num_parts_to_preprocess_at_once), 
                                   desc="Transforming transactions data"):
        transactions_frame = read_parquet_dataset_from_local(path_to_dataset, step, num_parts_to_preprocess_at_once, 
                                                             verbose=True)
        for dense_col in ['amnt', 'days_before', 'hour_diff']:
            transactions_frame[dense_col] = np.digitize(transactions_frame[dense_col], bins=dense_features_buckets[dense_col])
            
        seq = transform_transactions_to_sequences(transactions_frame)
        seq['sequence_length'] = seq.sequences.apply(lambda x: len(x[1]))
        
        if frame_with_ids is not None:
            seq = seq.merge(frame_with_ids, on='app_id')

        block_as_str = str(block)
        if len(block_as_str) == 1:
            block_as_str = '00' + block_as_str
        else:
            block_as_str = '0' + block_as_str
            
        processed_fragment =  create_padded_buckets(seq, mapping_seq_len_to_padded_len, has_target=has_target, 
                                                    save_to_file_path=os.path.join(save_to_path, 
                                                                                   f'processed_chunk_{block_as_str}.pkl'))
        block += 1

* Разобьем имеющиеся данные на `train` и `val` части. Воспользуемся самым простым способом - для валидации используем 10% случайных данных

In [7]:
train, val = train_test_split(target_frame, random_state=42, test_size=0.1)
train.shape, val.shape

((867429, 3), (96382, 3))

In [9]:
! rm -r ../../../val_buckets
! mkdir ../../../val_buckets

In [10]:
create_buckets_from_transactions(TRAIN_TRANSACTIONS_PATH, 
                                save_to_path='../../../val_buckets',
                                frame_with_ids=val, num_parts_to_preprocess_at_once=10, num_parts_total=50, has_target=True)

Transforming transactions data:   0%|          | 0/5 [00:00<?, ?it/s]

Reading chunks:

../../../train_transactions_contest/part_000_0_to_23646.parquet
../../../train_transactions_contest/part_001_23647_to_47415.parquet
../../../train_transactions_contest/part_002_47416_to_70092.parquet
../../../train_transactions_contest/part_003_70093_to_92989.parquet
../../../train_transactions_contest/part_004_92990_to_115175.parquet
../../../train_transactions_contest/part_005_115176_to_138067.parquet
../../../train_transactions_contest/part_006_138068_to_159724.parquet
../../../train_transactions_contest/part_007_159725_to_180735.parquet
../../../train_transactions_contest/part_008_180736_to_202834.parquet
../../../train_transactions_contest/part_009_202835_to_224283.parquet


../dataset_preprocessing_utils.py:104: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  'padded_sequences': np.array(padded_seq),
../dataset_preprocessing_utils.py:105: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  'targets': np.array(targets) if targets else [],
../dataset_preprocessing_utils.py:106: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the n

Reading chunks:

../../../train_transactions_contest/part_010_224284_to_245233.parquet
../../../train_transactions_contest/part_011_245234_to_265281.parquet
../../../train_transactions_contest/part_012_265282_to_285632.parquet
../../../train_transactions_contest/part_013_285633_to_306877.parquet
../../../train_transactions_contest/part_014_306878_to_329680.parquet
../../../train_transactions_contest/part_015_329681_to_350977.parquet
../../../train_transactions_contest/part_016_350978_to_372076.parquet
../../../train_transactions_contest/part_017_372077_to_392692.parquet
../../../train_transactions_contest/part_018_392693_to_413981.parquet
../../../train_transactions_contest/part_019_413982_to_434478.parquet


Transforming transactions data:  40%|████      | 2/5 [03:58<05:55, 118.66s/it]

...

Transforming transactions data:  60%|██████    | 3/5 [05:52<03:54, 117.29s/it]

Reading chunks:

../../../train_transactions_contest/part_030_635005_to_654605.parquet
../../../train_transactions_contest/part_031_654606_to_673656.parquet
../../../train_transactions_contest/part_032_673657_to_696025.parquet
../../../train_transactions_contest/part_033_696026_to_714545.parquet
../../../train_transactions_contest/part_034_714546_to_733168.parquet
../../../train_transactions_contest/part_035_733169_to_752514.parquet
../../../train_transactions_contest/part_036_752515_to_770940.parquet
../../../train_transactions_contest/part_037_770941_to_788380.parquet
../../../train_transactions_contest/part_038_788381_to_805771.parquet
../../../train_transactions_contest/part_039_805772_to_823299.parquet


Transforming transactions data:  80%|████████  | 4/5 [07:45<01:55, 115.86s/it]

Reading chunks:

../../../train_transactions_contest/part_040_823300_to_841218.parquet
../../../train_transactions_contest/part_041_841219_to_859270.parquet
../../../train_transactions_contest/part_042_859271_to_878521.parquet
../../../train_transactions_contest/part_043_878522_to_896669.parquet
../../../train_transactions_contest/part_044_896670_to_916056.parquet
../../../train_transactions_contest/part_045_916057_to_935131.parquet
../../../train_transactions_contest/part_046_935132_to_951695.parquet
../../../train_transactions_contest/part_047_951696_to_970383.parquet
../../../train_transactions_contest/part_048_970384_to_987313.parquet
../../../train_transactions_contest/part_049_987314_to_1003050.parquet


Transforming transactions data: 100%|██████████| 5/5 [09:36<00:00, 115.22s/it]


In [11]:
path_to_dataset = '../../../val_buckets'
dir_with_datasets = os.listdir(path_to_dataset)
dataset_val = sorted([os.path.join(path_to_dataset, x) for x in dir_with_datasets])
dataset_val

['../../../val_buckets/processed_chunk_000.pkl',
 '../../../val_buckets/processed_chunk_001.pkl',
 '../../../val_buckets/processed_chunk_002.pkl',
 '../../../val_buckets/processed_chunk_003.pkl',
 '../../../val_buckets/processed_chunk_004.pkl']

In [12]:
! rm -r ../../../train_buckets
! mkdir ../../../train_buckets

In [14]:
create_buckets_from_transactions(TRAIN_TRANSACTIONS_PATH, 
                                save_to_path='../../../train_buckets',
                                frame_with_ids=train, num_parts_to_preprocess_at_once=5, num_parts_total=50, has_target=True)

Transforming transactions data:   0%|          | 0/10 [00:00<?, ?it/s]

Reading chunks:

../../../train_transactions_contest/part_000_0_to_23646.parquet
../../../train_transactions_contest/part_001_23647_to_47415.parquet
../../../train_transactions_contest/part_002_47416_to_70092.parquet
../../../train_transactions_contest/part_003_70093_to_92989.parquet
../../../train_transactions_contest/part_004_92990_to_115175.parquet


Transforming transactions data:  10%|█         | 1/10 [01:25<12:50, 85.56s/it]

Reading chunks:

../../../train_transactions_contest/part_005_115176_to_138067.parquet
../../../train_transactions_contest/part_006_138068_to_159724.parquet
../../../train_transactions_contest/part_007_159725_to_180735.parquet
../../../train_transactions_contest/part_008_180736_to_202834.parquet
../../../train_transactions_contest/part_009_202835_to_224283.parquet


Transforming transactions data:  20%|██        | 2/10 [02:54<11:32, 86.55s/it]

...

Transforming transactions data:  80%|████████  | 8/10 [11:16<02:46, 83.39s/it]

Reading chunks:

../../../train_transactions_contest/part_040_823300_to_841218.parquet
../../../train_transactions_contest/part_041_841219_to_859270.parquet
../../../train_transactions_contest/part_042_859271_to_878521.parquet
../../../train_transactions_contest/part_043_878522_to_896669.parquet
../../../train_transactions_contest/part_044_896670_to_916056.parquet


Transforming transactions data:  90%|█████████ | 9/10 [12:43<01:24, 84.29s/it]

Reading chunks:

../../../train_transactions_contest/part_045_916057_to_935131.parquet
../../../train_transactions_contest/part_046_935132_to_951695.parquet
../../../train_transactions_contest/part_047_951696_to_970383.parquet
../../../train_transactions_contest/part_048_970384_to_987313.parquet
../../../train_transactions_contest/part_049_987314_to_1003050.parquet


Transforming transactions data: 100%|██████████| 10/10 [14:03<00:00, 84.34s/it]


In [15]:
path_to_dataset = '../../../train_buckets'
dir_with_datasets = os.listdir(path_to_dataset)
dataset_train = sorted([os.path.join(path_to_dataset, x) for x in dir_with_datasets])
dataset_train

['../../../train_buckets/processed_chunk_000.pkl',
 '../../../train_buckets/processed_chunk_001.pkl',
 '../../../train_buckets/processed_chunk_002.pkl',
 '../../../train_buckets/processed_chunk_003.pkl',
 '../../../train_buckets/processed_chunk_004.pkl',
 '../../../train_buckets/processed_chunk_005.pkl',
 '../../../train_buckets/processed_chunk_006.pkl',
 '../../../train_buckets/processed_chunk_007.pkl',
 '../../../train_buckets/processed_chunk_008.pkl',
 '../../../train_buckets/processed_chunk_009.pkl']

### 2. Modeling

* Для создания модели будем использовать фреймворк `tensorflow`. В нем есть все, чтобы писать произвольные сложные архитектуры и быстро эксперементировать.

* В бейзлайне мы предлагаем базовые компоненты, чтобы можно было обучать нейронную сеть и отслеживать ее качество. Для этого вам предоставлены следующие функции:
    * `data_generators.batches_generator` - функция-генератор, итеративно возвращает батчи, поддерживает батчи для `tensorflow.keras` и `torch.nn.module` моделей. В зависимости от флага `is_train` может быть использована для генерации батчей на train/val/test стадию.
    * функция `tf_training.train_epoch` - обучает модель одну эпоху.
    * функция `tf_training.eval_model` - проверяет качество модели на отложенной выборке и возвращает roc_auc_score.
    * функция `tf_training.inference` - делает предикты на новых данных и готовит фрейм для проверяющей системы.
    * класс `training_aux.EarlyStopping` - реализует early_stopping, сохраняя лучшую модель. Пример использования приведен ниже.

In [16]:
from data_generators import batches_generator, transaction_features
from tf_training import train_epoch, eval_model, inference
from training_aux import EarlyStopping

* Все признаки в нашей модели будут категориальными. Для их представления в модели используем категориальные эмбеддинги. Для этого нужно каждому категориальному признаку задать размерность латентного пространства. Используем [формулу](https://forums.fast.ai/t/size-of-embedding-for-categorical-variables/42608) из библиотеки `fast.ai`. Все отображения хранятся в файле `embedding_projections.pkl`

In [17]:
with open('../constants/embedding_projections.pkl', 'rb') as f:
    embedding_projections = pickle.load(f)

* Реализуем модель. Все входные признаки представим в виде эмбеддингов, сконкатенируем, чтобы получить векторное представление транзакции. Подадим последовательности в `GRU` рекуррентную сеть. Используем последнее скрытое состояние в качестве выхода сети. Представим признак `product` в виде отдельного эмбеддинга. Сконкатенируем его с выходом сети. На основе такого входа построим небольшой `MLP`, выступающий классификатором для целевой задачи. Используем градиентный спуск, чтобы решить оптимизационную задачу. 

In [18]:
def build_transactions_rnn(transactions_cat_features, embedding_projections, product_col_name='product', 
                          rnn_units=128, classifier_units=32, optimizer=None):
    if not optimizer:
        optimizer = keras.optimizers.Adam(lr=1e-3)
        
    inputs = []
    cat_embeds = []
    
    for feature_name in transactions_cat_features:
        inp = L.Input(shape=(None, ), dtype='uint32', name=f'input_{feature_name}')
        inputs.append(inp)
        source_size, projection = embedding_projections[feature_name]
        emb = L.Embedding(source_size+1, projection, trainable=True, mask_zero=False, name=f'embedding_{feature_name}')(inp)
        cat_embeds.append(emb)
    
    # product feature
    inp = L.Input(shape=(1, ), dtype='uint32', name=f'input_product')
    inputs.append(inp)
    source_size, projection = embedding_projections['product']
    product_emb = L.Embedding(source_size+1, projection, trainable=True, mask_zero=False, name=f'embedding_product')(inp)
    product_emb_reshape = L.Reshape((projection, ))(product_emb)
    
    concated_cat_embeds = L.concatenate(cat_embeds)
    
    last_state = L.GRU(units=rnn_units)(concated_cat_embeds)
    
    combined_inp = L.concatenate([last_state, product_emb_reshape])
    
    dense_intermediate = L.Dense(classifier_units, activation='relu', )(combined_inp)
    proba = L.Dense(1, activation='sigmoid')(dense_intermediate)
    
    model = Model(inputs=inputs, outputs=proba)
    model.compile(loss='binary_crossentropy', optimizer=optimizer)
    return model

### 3. Training

In [19]:
! mkdir ../../rnn_baseline/checkpoints/

mkdir: cannot create directory ‘../../rnn_baseline/checkpoints/’: File exists


In [20]:
! rm -r ../../rnn_baseline/checkpoints/tf_baseline
! mkdir ../../rnn_baseline/checkpoints/tf_baseline

* Для того, чтобы детектировать переобучение используем EarlyStopping.

In [21]:
path_to_checkpoints = '../../rnn_baseline/checkpoints/tf_baseline/'
es = EarlyStopping(patience=3, mode='max', verbose=True, save_path=os.path.join(path_to_checkpoints, 'best_checkpoint.pt'), 
                   metric_name='ROC-AUC', save_format='tf')

In [22]:
num_epochs = 15
train_batch_size = 128
val_batch_szie = 128

In [23]:
model = build_transactions_rnn(transaction_features, embedding_projections)

In [24]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_currency (InputLayer)     [(None, None)]       0                                            
__________________________________________________________________________________________________
input_operation_kind (InputLaye [(None, None)]       0                                            
__________________________________________________________________________________________________
input_card_type (InputLayer)    [(None, None)]       0                                            
__________________________________________________________________________________________________
input_operation_type (InputLaye [(None, None)]       0                                            
______________________________________________________________________________________________

* Запустим цикл обучения, каждую эпоху будем логировать лосс, а так же roc-auc на валидации и на обучении. Будем сохрнаять веса после каждой эпохи, а так же лучшие с помощью early_stopping.

In [25]:
for epoch in range(num_epochs):
    print(f'Starting epoch {epoch+1}')
    train_epoch(model, dataset_train, batch_size=train_batch_size, shuffle=True, cur_epoch=epoch, 
                steps_per_epoch=7270)
    
    val_roc_auc = eval_model(model, dataset_val, batch_size=val_batch_szie)
    model.save_weights(os.path.join(path_to_checkpoints, f'epoch_{epoch+1}_val_{val_roc_auc:.3f}.hdf5'))
    
    es(val_roc_auc, model)
    
    if es.early_stop:
        print('Early stopping reached. Stop training...')
        break
        
    train_roc_auc = eval_model(model, dataset_train, batch_size=val_batch_szie)
    print(f'Epoch {epoch+1} completed. Train roc-auc: {train_roc_auc}, Val roc-auc: {val_roc_auc}')

Starting epoch 1
7270/7270 [==============================] - 181s 24ms/step - loss: 0.1258
Validation ROC-AUC improved (-inf --> 0.770105).  Saving model ...
Epoch 1 completed. Train roc-auc: 0.7707000528052049, Val roc-auc: 0.7701051844520257
Starting epoch 2
Epoch 2/2
7270/7270 [==============================] - 174s 24ms/step - loss: 0.1114
Validation ROC-AUC improved (0.770105 --> 0.779645).  Saving model ...
Epoch 2 completed. Train roc-auc: 0.7837507252665802, Val roc-auc: 0.779644731601884
Starting epoch 3
Epoch 3/3
7270/7270 [==============================] - 174s 24ms/step - loss: 0.1096
Validation ROC-AUC improved (0.779645 --> 0.786627).  Saving model ...
Epoch 3 completed. Train roc-auc: 0.7993362070628552, Val roc-auc: 0.7866268862634636
Starting epoch 4
Epoch 4/4
7270/7270 [==============================] - 174s 24ms/step - loss: 0.1082
Validation ROC-AUC improved (0.786627 --> 0.792755).  Saving model ...
Epoch 4 completed. Train roc-auc: 0.8117309906418221, Val roc-auc

### 4. Submission

* Все готово, чтобы сделать предсказания для тестовой выборки. Нужно только подготовить данные в том же формате, как и для train.

In [66]:
! rm -r ../../../test_buckets
! mkdir ../../../test_buckets

In [66]:
test_frame = pd.read_csv('../../../test_target_contest.csv')
test_frame.head()

,app_id,product
0,1063620,0
1,1063621,0
2,1063622,1
3,1063623,1
4,1063624,2


In [68]:
create_buckets_from_transactions(TEST_TRANSACTIONS_PATH, 
                                save_to_path='../../../test_buckets', frame_with_ids=test_frame, 
                                 num_parts_to_preprocess_at_once=10, num_parts_total=50, has_target=False)

Transforming transactions data:   0%|          | 0/5 [00:00<?, ?it/s]

Reading chunks:

../../../test_transactions_contest/part_000_1063620_to_1074462.parquet
../../../test_transactions_contest/part_001_1074463_to_1085303.parquet
../../../test_transactions_contest/part_002_1085304_to_1095174.parquet
../../../test_transactions_contest/part_003_1095175_to_1105002.parquet
../../../test_transactions_contest/part_004_1105003_to_1116054.parquet
../../../test_transactions_contest/part_005_1116055_to_1127527.parquet
../../../test_transactions_contest/part_006_1127528_to_1137672.parquet
../../../test_transactions_contest/part_007_1137673_to_1147504.parquet
../../../test_transactions_contest/part_008_1147505_to_1157749.parquet
../../../test_transactions_contest/part_009_1157750_to_1167980.parquet


../dataset_preprocessing_utils.py:104: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  'padded_sequences': np.array(padded_seq),
../dataset_preprocessing_utils.py:106: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  'app_id': np.array(app_ids),
../dataset_preprocessing_utils.py:107: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  'products':

Reading chunks:

../../../test_transactions_contest/part_010_1167981_to_1178851.parquet
../../../test_transactions_contest/part_011_1178852_to_1190630.parquet
../../../test_transactions_contest/part_012_1190631_to_1200939.parquet
../../../test_transactions_contest/part_013_1200940_to_1211425.parquet
../../../test_transactions_contest/part_014_1211426_to_1222122.parquet
../../../test_transactions_contest/part_015_1222123_to_1232298.parquet
../../../test_transactions_contest/part_016_1232299_to_1242388.parquet
../../../test_transactions_contest/part_017_1242389_to_1252416.parquet
../../../test_transactions_contest/part_018_1252417_to_1262614.parquet
../../../test_transactions_contest/part_019_1262615_to_1273376.parquet


Transforming transactions data:  40%|████      | 2/5 [03:33<05:20, 106.93s/it]

...

Transforming transactions data:  60%|██████    | 3/5 [05:17<03:31, 105.90s/it]

Reading chunks:

../../../test_transactions_contest/part_030_1376992_to_1386419.parquet
../../../test_transactions_contest/part_031_1386420_to_1395884.parquet
../../../test_transactions_contest/part_032_1395885_to_1405390.parquet
../../../test_transactions_contest/part_033_1405391_to_1416489.parquet
../../../test_transactions_contest/part_034_1416492_to_1426763.parquet
../../../test_transactions_contest/part_035_1426764_to_1436400.parquet
../../../test_transactions_contest/part_036_1436401_to_1448080.parquet
../../../test_transactions_contest/part_037_1448081_to_1459730.parquet
../../../test_transactions_contest/part_038_1459731_to_1470134.parquet
../../../test_transactions_contest/part_039_1470135_to_1479802.parquet


Transforming transactions data:  80%|████████  | 4/5 [06:57<01:44, 104.24s/it]

Reading chunks:

../../../test_transactions_contest/part_040_1479803_to_1489232.parquet
../../../test_transactions_contest/part_041_1489233_to_1499712.parquet
../../../test_transactions_contest/part_042_1499713_to_1510447.parquet
../../../test_transactions_contest/part_043_1510448_to_1520793.parquet
../../../test_transactions_contest/part_044_1520794_to_1531282.parquet
../../../test_transactions_contest/part_045_1531283_to_1541445.parquet
../../../test_transactions_contest/part_046_1541446_to_1551040.parquet
../../../test_transactions_contest/part_047_1551041_to_1560328.parquet
../../../test_transactions_contest/part_048_1560329_to_1570341.parquet
../../../test_transactions_contest/part_049_1570342_to_1580442.parquet


Transforming transactions data: 100%|██████████| 5/5 [08:37<00:00, 103.59s/it]


In [68]:
path_to_test_dataset = '../../../test_buckets/'
dir_with_test_datasets = os.listdir(path_to_test_dataset)
dataset_test = sorted([os.path.join(path_to_test_dataset, x) for x in dir_with_test_datasets])

dataset_test

['../../../test_buckets/processed_chunk_000.pkl',
 '../../../test_buckets/processed_chunk_001.pkl',
 '../../../test_buckets/processed_chunk_002.pkl',
 '../../../test_buckets/processed_chunk_003.pkl',
 '../../../test_buckets/processed_chunk_004.pkl']

* Отдельный вопрос, какую из построенных моделей использовать для того, чтобы делать предсказания на тест. Можно выбирать лучшую по early_stopping. В таком случае есть риск, что мы подгонимся под валидационную выборку, особенно если она не является очень репрезентативной, однако это самый базовый вариант (используем его). Можно делать разные версии ансамблирования, используя веса с разных эпох. Такой подход требует дополнительного кода (обязательно попробуйте его!). Наконец, можно выбирать такую модель, которая показывает хорошие результаты на валидации и в то же время, не слишком переучена под train выборку.

In [69]:
! ls $path_to_checkpoints

best_checkpoint.pt.data-00000-of-00001	epoch_3_val_0.787.hdf5
best_checkpoint.pt.index		epoch_4_val_0.793.hdf5
checkpoint				epoch_5_val_0.791.hdf5
epoch_1_val_0.770.hdf5			epoch_6_val_0.789.hdf5
epoch_2_val_0.780.hdf5			epoch_7_val_0.788.hdf5


In [97]:
model.load_weights(os.path.join(path_to_checkpoints, 'best_checkpoint.pt'))

In [98]:
test_preds = inference(model, dataset_test, batch_size=128)

In [99]:
test_preds.head()

,app_id,score
0,1063655,0.037216
1,1063672,0.041969
2,1063694,0.019908
3,1063709,0.050075
4,1063715,0.040230


In [102]:
test_preds.to_csv('rnn_baseline_submission.csv', index=None) # ~ 0.750 на public test